In [8]:
import json
from pymorphy2 import MorphAnalyzer
from tqdm import tqdm
import pandas as pd
import numpy as np
import random
import nltk

In [7]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [10]:
with open('corpus.json', encoding='utf-8') as f:
    corpus = json.load(f)

In [11]:
'Текстов - {}, слов - {}'.format(len(corpus), sum([len(sample) for sample in corpus]))

'Текстов - 1157366, слов - 16028874'

In [12]:
morph = MorphAnalyzer()

In [13]:
def get_lemma(word):
    
    word_data = morph.parse(word)[0]
    
    return word_data.normal_form

## Соберем словарь лем
В нашем корпусе 16028874 слов. Лемматизировать весь корпус будет очень долго. Давайте лучше соберем словарь уникальных слов и будет лемматизировать только уникальные слова.

In [14]:
tok2lemma = {}

for text in tqdm(corpus):
    for tok in text:
        if tok not in tok2lemma:
            tok2lemma[tok] = get_lemma(tok)

100%|█████████████████████████████████████████████████████████████████████| 1157366/1157366 [00:36<00:00, 32105.18it/s]


In [17]:
# ключ - уникальное слово в нашем корпусе, значение - его лемма
tok2lemma['уехали']

'уехать'

In [18]:
len(tok2lemma)

193435

In [17]:
stopwords = nltk.corpus.stopwords.words('russian')

# Замена слов леммами
Так как теперь к каждому слову из нашего корпуса мы знаем лемму, то давайте каждое слово заменим на его лемму и уберем стоп слова.
Это работает гораздо(!) быстрее, чем если бы мы в корпусе для каждого слова каждый раз рассчитывали лемму (с помощью пайморфи), 
потому что теперь нам надо вызвать пайморфи 193435 раз вместо 16028874.

In [18]:
lemmas_corpus = [[tok2lemma[tok] for tok in text if tok not in stopwords and tok]
                 for text in tqdm(corpus)]

100%|█████████████████████████████████████████████████████████████████████| 1157366/1157366 [00:28<00:00, 40151.31it/s]


# Соберем частотный словарь

In [19]:
freq = {}

for text in tqdm(lemmas_corpus):
    for tok in text:
        if tok in freq:
            freq[tok] += 1
        else:
            freq[tok] = 1

100%|████████████████████████████████████████████████████████████████████| 1157366/1157366 [00:03<00:00, 382845.21it/s]


In [20]:
freq_df = pd.DataFrame(data={'word': list(freq.keys()), 'n_entries': list(freq.values())})

In [21]:
freq_df.sort_values(by=['n_entries'], ascending=False, inplace=True)

In [22]:
freq_df.head()

,word,n_entries
21,##число,413016
3,банк,333814
46,карта,156216
134,кредит,86865
100,это,81577


In [25]:
# уникальных слов в словаре
freq_df.shape

(76916, 2)

In [26]:
freq_df.tail()

,word,n_entries
21814,приборный,1
21813,подписнуть,1
21812,позволинь,1
50667,marinaurievnaобещание,1
76915,ситуевина,1


In [23]:
n_words = freq_df.n_entries.sum()

# Замена редких слов
В нашем корпусе осталось много слов, которые встречаются очень редко. Давайте мы редкие слова заменим на специальный токе UNK - unknown. Так мы разительно сократим размер нашего словаря слов с незначительной потерей информации.

In [24]:
print('Доля слов, которые мы заменим на UNK:')

for threshold in np.arange(5, 36, 5):
    
    sub_df = freq_df[freq_df.n_entries < threshold]
    
    unk_freq = sub_df['n_entries'].sum() * 100 / n_words
    
    print('Порог отсечения - {}, доля UNK - {:.2f} %, слов в слове - {}, удалили - {} слов'.format(
        threshold, unk_freq, freq_df.shape[0] - sub_df.shape[0], sub_df.shape[0]))

Доля слов, которые мы заменим на UNK:
Порог отсечения - 5, доля UNK - 0.76 %, слов в слове - 23307, удалили - 53609 слов
Порог отсечения - 10, доля UNK - 1.18 %, слов в слове - 16771, удалили - 60145 слов
Порог отсечения - 15, доля UNK - 1.51 %, слов в слове - 13782, удалили - 63134 слов
Порог отсечения - 20, доля UNK - 1.81 %, слов в слове - 11946, удалили - 64970 слов
Порог отсечения - 25, доля UNK - 2.07 %, слов в слове - 10737, удалили - 66179 слов
Порог отсечения - 30, доля UNK - 2.30 %, слов в слове - 9835, удалили - 67081 слов
Порог отсечения - 35, доля UNK - 2.52 %, слов в слове - 9135, удалили - 67781 слов


In [25]:
# кажется, что оптимально, но обычно берут меньше
threshold = 15

In [26]:
vocab = freq_df[freq_df.n_entries >= threshold]

In [27]:
words = set(vocab.word)

In [28]:
len(words)

13782

In [29]:
'Мы сократили наш словарь в {:.2f} раз с потерей 1.51 % всех слов'.format(freq_df.shape[0] / len(words))

'Мы сократили наш словарь в 5.58 раз с потерей 1.51 % всех слов'

In [30]:
def get_correct_words(word):
    
    if word in words:
        return word
    else:
        return 'UNK'

In [31]:
# заменим слово токеном UNK, если его нет в нашем новом словаре
processed_corpus = [[get_correct_words(tok) for tok in text] for text in tqdm(lemmas_corpus)]

100%|████████████████████████████████████████████████████████████████████| 1157366/1157366 [00:03<00:00, 325806.87it/s]


In [32]:
def drop_duplicate_unks(tokens):
    
    output_tokens = []
    
    for tok in tokens:
        
        if tok == 'UNK' and output_tokens and output_tokens[-1] == 'UNK':
            continue
            
        output_tokens.append(tok)
            
    return output_tokens

In [33]:
sample_text = 'думать далее милый барышня UNK UNK тинькоф звонить неделя'.split()

In [34]:
sample_text

['думать',
 'далее',
 'милый',
 'барышня',
 'UNK',
 'UNK',
 'тинькоф',
 'звонить',
 'неделя']

In [35]:
drop_duplicate_unks(sample_text)

['думать', 'далее', 'милый', 'барышня', 'UNK', 'тинькоф', 'звонить', 'неделя']

In [36]:
# дедублируем подряд идущие унки (оставим только один)
processed_corpus = [drop_duplicate_unks(sample) for sample in tqdm(processed_corpus)]

100%|████████████████████████████████████████████████████████████████████| 1157366/1157366 [00:03<00:00, 302184.81it/s]


In [37]:
texts_with_unk = [text for text in processed_corpus if 'UNK' in text]
'Текстов с унками - {:.2f} %'.format(len(texts_with_unk) * 100 / len(processed_corpus))

'Текстов с унками - 11.27 %'

In [38]:
# посмотрим на тексты с унками
for text in random.sample(texts_with_unk, k=5):
    print(' '.join(text))

получаться сотрудник оао UNK россия производить выпуск карта ведомо UNK
вчера вечером поступить смс момент посещение UNK адрес проживание
повторный звонок алексей ответить этот информация слово печать UNK деньга
читать человек несколько день банкомат свой вклад ткс UNK
кроме весь договор достаточно UNK напечатать мелкий шрифт целый ##число лист


In [43]:
# выглядит не так плохо

In [39]:
random.shuffle(processed_corpus)

# Выберем подвыборку данных
Чтобы быстрее выучить word2vec

In [40]:
sub_data = processed_corpus[:175000]

In [42]:
with open('processed_corpus2.json', 'w') as f:
    json.dump(sub_data, f, ensure_ascii=False)